In [ ]:
!pip install -q transformers accelerate torch sentencepiece

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [ ]:
model_names = {
    "distilgpt2": "distilbert/distilgpt2",
    "gpt_oss_20b": "openai/gpt-oss-20b"
}

In [ ]:
tokenizers = {}
models = {}

for apelido, nome in model_names.items():
    print(f"\n=== Carregando {apelido} ({nome}) ===")
    # Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(nome)

    # Alguns modelos não têm pad_token definido
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    tokenizers[apelido] = tokenizer

    # Modelo
    if apelido == "gpt_oss_20b":
        # Modelo grande: usar device_map e dtype reduzido
        model = AutoModelForCausalLM.from_pretrained(
            nome,
            torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            device_map="auto"
        )
    else:
        # Modelo menor: pode ir direto pra GPU se tiver
        model = AutoModelForCausalLM.from_pretrained(nome)
        model = model.to(device)

    models[apelido] = model

print("\nModelos carregados com sucesso (se não deu erro acima 😊).")



=== Carregando distilgpt2 (distilbert/distilgpt2) ===


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]


Modelos carregados com sucesso (se não deu erro acima 😊).


In [ ]:
def gerar_texto(model_key, prompt, max_new_tokens=80, temperature=0.8, top_p=0.9):
    tokenizer = tokenizers[model_key]
    model = models[model_key]

    # Ensure input tensor dtype matches model dtype
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device).to(model.dtype)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=top_p,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )

    texto = tokenizer.decode(output[0], skip_special_tokens=True)
    return texto


def comparar_modelos(prompt, max_new_tokens=80):
    resultados = {}
    for apelido in model_names.keys():
        try:
            print(f"\n>>> Gerando com modelo: {apelido}")
            resposta = gerar_texto(
                model_key=apelido,
                prompt=prompt,
                max_new_tokens=max_new_tokens
            )
            resultados[apelido] = resposta
        except Exception as e:
            resultados[apelido] = f"Erro ao gerar com {apelido}: {e}"

    return resultados

In [ ]:
def analisar_resultados(prompt, resultados):
    print("\nAnálise simples das respostas:")
    for modelo, texto in resultados.items():
        num_caracteres = len(texto)
        num_palavras = len(texto.split())
        print(f"- {modelo}: {num_caracteres} caracteres, {num_palavras} palavras.")

# Definir a lista de prompts antes de usá-la
prompts = [
    "Qual é a capital da França?",
    "Me conte uma história curta sobre um robô que se apaixona."
]

# Exemplo usando apenas o primeiro prompt:
prompt_teste = prompts[0]
print("Prompt de teste:", prompt_teste)

resultados_teste = comparar_modelos(prompt_teste, max_new_tokens=80)
for modelo, resposta in resultados_teste.items():
    print(f"\n[{modelo}]")
    print(resposta[:400], "\n")  # corta pra não ficar enorme

analisar_resultados(prompt_teste, resultados_teste)

Attempting to cast a BatchEncoding to type torch.float32. This is not supported.


Prompt de teste: Qual é a capital da França?

>>> Gerando com modelo: distilgpt2


Attempting to cast a BatchEncoding to type torch.float32. This is not supported.



>>> Gerando com modelo: gpt_oss_20b

[distilgpt2]
Qual é a capital da França?















































































 


[gpt_oss_20b]
Erro ao gerar com gpt_oss_20b: expected m1 and m2 to have the same dtype, but got: float != c10::BFloat16 


Análise simples das respostas:
- distilgpt2: 107 caracteres, 6 palavras.
- gpt_oss_20b: 105 caracteres, 19 palavras.
